In [1]:
from utils.google_trans_new import google_translator

translator = google_translator()

test = translator.translate("สวัสดีจีน", lang_tgt="en")
print(test)
txt_to_translate = "hello world"
language = "fr"
translation = translator.translate(txt_to_translate, lang_tgt=language)
print(translation)

Hello Chinese 
Bonjour le monde 


In [2]:
# coolio, now we have a way of automatically translating text. let's do that for each language we have available.
import glob
import json

data_dir = "../webapp/data/"
language_codes = [f.split("/")[-1] for f in glob.glob(f"{data_dir}/languages/*")]

with open(f"{data_dir}/languages.json", "r") as f:
    languages = json.load(f)

print(languages["en"])

{'language_code': 'en', 'language_name': 'English', 'language_name_native': 'English', 'language_emoji': '🇪🇳'}


In [3]:
import time


def translate_text(txt_to_translate, language):
    try:
        translation = translator.translate(txt_to_translate, lang_tgt=language)
    except:
        print(f"error translating text: {txt_to_translate} lang: {language}")
        time.sleep(0.5)
        return txt_to_translate
    # sleep half a sec
    time.sleep(0.5)
    return translation


translate_text("hackernews is fun", "de")

'Hackernews macht Spaß '

In [4]:
# load english language config
with open(f"{data_dir}/languages/en/language_config.json", "r") as f:
    en_config = json.load(f)

f"{data_dir}/languages/en/language_config.json"

'../webapp/data//languages/en/language_config.json'

In [5]:
# make it into reusable function
import os


def translate_language(from_language, to_language):
    """translates a language config from one language to another
    Caution: overwrites any existing language config, and keyboard is left empty
    (it's hard to determine keyboard layout programmatically, so sorting it is best option)"""

    # load from_language config
    with open(
        f"{data_dir}languages/{from_language}/language_config.json",
        "r",
        encoding="utf-8",
    ) as f:
        from_language_config = json.load(f)

    # if existing config exists load keyboard from it
    to_keyboard = ""
    if os.path.exists(f"{data_dir}languages/{to_language}/language_config.json"):
        with open(
            f"{data_dir}languages/{to_language}/language_config.json",
            "r",
            encoding="utf-8",
        ) as f:
            to_language_config = json.load(f)
            to_keyboard = to_language_config["keyboard"]

    # write to_language config
    with open(
        f"{data_dir}languages/{to_language}/language_config.json", "w", encoding="utf-8"
    ) as f:
        language_config = {}
        language_config["language_code"] = to_language
        language_config["name"] = languages[to_language]["language_name"]
        language_config["name_native"] = languages[to_language]["language_name_native"]
        language_config["meta"] = {}
        language_config["meta"]["locale"] = to_language
        language_config["text"] = {}
        language_config["text"]["subheader"] = languages[to_language][
            "language_name_native"
        ]

        # defaults
        language_config["language_code_3"] = ""
        language_config["language_code_iso_639_3"] = ""

        language_config["keyboard"] = to_keyboard

        right_to_left_languages = ["ar", "fa", "he", "ur"]
        if to_language in right_to_left_languages:
            language_config["right_to_left"] = "true"
        else:
            language_config["right_to_left"] = "false"

        # translate these
        language_config["meta"]["title"] = translate_text(
            from_language_config["meta"]["title"], to_language
        )
        language_config["meta"]["description"] = translate_text(
            from_language_config["meta"]["description"], to_language
        )
        language_config["meta"]["keywords"] = translate_text(
            from_language_config["meta"]["keywords"], to_language
        )
        language_config["text"]["next_word"] = translate_text(
            from_language_config["text"]["next_word"], to_language
        )
        language_config["text"]["no_attempts"] = translate_text(
            from_language_config["text"]["no_attempts"], to_language
        )
        language_config["text"]["share"] = translate_text(
            from_language_config["text"]["share"], to_language
        )
        language_config["text"]["notification-copied"] = translate_text(
            from_language_config["text"]["notification-copied"], to_language
        )
        language_config["text"]["notification-partial-word"] = translate_text(
            from_language_config["text"]["notification-partial-word"], to_language
        )

        language_config["help"] = {}
        for key in from_language_config["help"]:
            translated_text = translate_text(
                from_language_config["help"][key], to_language
            )
            if key in [
                "text_2_1",
                "text_2_2",
                "text_2_3",
            ]:  # lowercase the first letter for these (looks better in help modal)
                translated_text = translated_text[0].lower() + translated_text[1:]
            language_config["help"][key] = translated_text

        json.dump(language_config, f, indent=4, ensure_ascii=False)
        print(f"Wrote language_config for {to_language}")

In [7]:
# for language_code in list(languages.keys())[47:]:
for language_code in languages:
    # check if the folder exists with glob
    if not glob.glob(f"{data_dir}/languages/{language_code}/*"):
        continue
    # skip manually checked languages
    if language_code in ["en", "ar"]:
        continue
    translate_language("en", language_code)

Wrote language_config for hy
Wrote language_config for eu
error translating text: The daily word game lang: br
error translating text: Guess the hidden word in 6 tries (or less). A new puzzle is available each day! lang: br
error translating text: english, puzzle, word, play, game, online, guess, daily lang: br
error translating text: Next word  lang: br
error translating text: You haven't tried any words yet!  lang: br
error translating text: Share  lang: br
error translating text: Copied to clipboard  lang: br
error translating text: Please enter a full word  lang: br
error translating text: How to Play lang: br
error translating text: Examples lang: br
error translating text: Close this message lang: br
error translating text: Guess the lang: br
error translating text: in six tries or less. lang: br
error translating text: Each guess must be a valid five letter word. Press the enter button to submit your guess. lang: br
error translating text: After you guess, the tiles will change 

In [6]:
translate_language("en", "ar")

Wrote language_config for ar


In [7]:
translate_language("en", "en")

Wrote language_config for en


In [1]:
# for each language,
# 1. load language_config.json and remove the keyboard
# 2. add a new file called keyboard.json

import glob
import json
import os

data_dir = "../webapp/data/"

for lang in glob.glob(f"{data_dir}languages/*"):
    lang = lang.split("/")[-1]
    try:
        with open(f"{data_dir}/languages/{lang}/language_config.json", "r") as f:
            language_config = json.load(f)

        with open(f"{data_dir}/languages/{lang}/keyboard.json", "w") as f:
            json.dump(language_config["keyboard"], f, indent=4, ensure_ascii=False)

        del language_config["keyboard"]
        with open(f"{data_dir}/languages/{lang}/language_config.json", "w") as f:
            json.dump(language_config, f, indent=4, ensure_ascii=False)
    except:
        print(f"Error with {lang}")

In [2]:
# rename all keyboard.json files to {language_code}_keyboard.json
for lang in glob.glob(f"{data_dir}languages/*"):
    lang = lang.split("/")[-1]
    try:
        os.rename(
            f"{data_dir}/languages/{lang}/keyboard.json",
            f"{data_dir}/languages/{lang}/{lang}_keyboard.json",
        )
    except:
        print(f"Error with {lang}")

In [ ]:
# modify all keyboard.json content from "" to []
for lang in glob.glob(f"{data_dir}languages/*"):
    lang = lang.split("/")[-1]
    try:
        with open(f"{data_dir}/languages/{lang}/{lang}_keyboard.json", "r") as f:
            keyboard = json.load(f)

        keyboard = []
        with open(f"{data_dir}/languages/{lang}/{lang}_keyboard.json", "w") as f:
            json.dump(keyboard, f, indent=4, ensure_ascii=False)
    except:
        print(f"Error with {lang}")